In [1]:
## Notice!! Please change the file path of following codes into your own
raw_files_path = '/Users/Weiqi0/ISB_working/Ilya_lab/Translator/Pharmagenomics_KG/files/SIGNOR/'

## Note! define download path in the very end of the notebook!!

In [2]:
## Check all node files being read
## Read all BigGIM node csv file in group 1
import os

for f in os.listdir(raw_files_path):
    if f.endswith('.tsv'):
        print(f)

all_data_11_09_25.tsv
all_data_24_09_25.tsv
all_data_11_12_25.tsv


In [3]:
import pandas as pd

## Read each individual csv files
source_df = pd.read_csv(raw_files_path + 'all_data_11_12_25.tsv', sep='\t')

## check the length
print("current row number is: ", len(source_df))

## Drop nan values
source_df = source_df.dropna(subset=['ENTITYA', 'ENTITYB'])

## rename those columns into desired format
source_df.rename(columns={'ENTITYA': 'subject_name', 'TYPEA': 'subject_category', 'ENTITYB': 'object_name', 'TYPEB': 'object_category'}, inplace=True)

## replace phenotype labeling into biologicalProcess
source_df["subject_category"] = source_df["subject_category"].replace("phenotype", "BiologicalProcess")
source_df ["object_category"] = source_df["object_category"].replace("phenotype", "BiologicalProcess")

## replace all 'miR-34' to 'miR-34a' in two columns subject_category and object_category in the pandas dataframe
source_df['subject_name'] = source_df['subject_name'].replace('miR-34', 'miR-34a')
source_df['object_name'] = source_df['object_name'].replace('miR-34', 'miR-34a')

## remove those rows with category in fusion protein or stimulus from source_df for now, and expecting biolink team to add those new categories
source_df = source_df[(source_df['subject_category'] != 'fusion Protein') & (source_df['object_category'] != 'fusion Protein')]
source_df = source_df[(source_df['subject_category'] != 'stimulus') & (source_df['object_category'] != 'stimulus')]

## check the length after filtering
print("current row number is: ", len(source_df))

source_df.head(10)

current row number is:  41575
current row number is:  41321


,subject_name,subject_category,IDA,DATABASEA,object_name,object_category,IDB,DATABASEB,EFFECT,MECHANISM,...,MODASEQ,MODIFICATIONB,MODBSEQ,PMID,DIRECT,NOTES,ANNOTATOR,SENTENCE,SCORE,SIGNOR_ID
0,EIF2B5,protein,Q13144,UNIPROT,EIF2S2,protein,P20042,UNIPROT,up-regulates activity,guanine nucleotide exchange factor,...,NaN,NaN,NaN,15054402,YES,NaN,lperfetto,EIF2B converts the protein synthesis initiatio...,0.877,SIGNOR-269128
1,PKA,proteinfamily,SIGNOR-PF17,SIGNOR,ABCB1,protein,P08183,UNIPROT,up-regulates activity,dephosphorylation,...,NaN,NaN,NaN,24333728,YES,NaN,NaN,Protein phosphatase complex PP5/PPP2R3C dephos...,0.200,SIGNOR-272508
2,cis-(z)-Flupenthixol,chemical,CHEBI:10454,ChEBI,DRD3,protein,P35462,UNIPROT,down-regulates activity,chemical inhibition,...,NaN,NaN,NaN,8301582,YES,NaN,miannu,The most selective compound from this group we...,0.800,SIGNOR-258715
3,CPT1C,protein,Q8TCG5,UNIPROT,coenzyme A(4-),smallmolecule,CHEBI:57287,ChEBI,up-regulates quantity,chemical modification,...,NaN,NaN,NaN,14517221,YES,NaN,miannu,Carnitine palmitoyltransferase 1A (CPT1A) is t...,0.800,SIGNOR-267134
4,TGFBR1,protein,P36897,UNIPROT,SPTBN1,protein,Q01082,UNIPROT,up-regulates,phosphorylation,...,NaN,NaN,NaN,12543979,YES,NaN,gcesareni,"This suggests that, upon stimulation with tgf-...",0.521,SIGNOR-97626
5,CDK5,protein,Q00535,UNIPROT,PIK3C3,protein,Q8NEB9,UNIPROT,down-regulates,phosphorylation,...,NaN,NaN,NaN,20513426,YES,NaN,llicata,Thr159 phosphorylation negatively regulates th...,0.357,SIGNOR-165772
6,SRC,protein,P12931,UNIPROT,HCN2,protein,Q9UL51,UNIPROT,up-regulates activity,phosphorylation,...,NaN,NaN,NaN,26280531,YES,NaN,miannu,We identified a highly conserved tyrosine resi...,0.268,SIGNOR-263199
7,BCL2L11,protein,O43521,UNIPROT,Apoptosis,BiologicalProcess,SIGNOR-PH2,SIGNOR,up-regulates,NaN,...,NaN,NaN,NaN,17960585,NO,NaN,miannu,Transforming growth factor-beta (TGF-beta) sig...,0.700,SIGNOR-260426
8,PTPRJ,protein,Q12913,UNIPROT,CTNNB1,protein,P35222,UNIPROT,up-regulates activity,dephosphorylation,...,NaN,NaN,NaN,25386896,YES,NaN,lperfetto,Our data demonstrate that CD148 promotes E-cad...,0.517,SIGNOR-276992
9,MFGE8,protein,Q08431,UNIPROT,Av/b3 integrin,complex,SIGNOR-C177,SIGNOR,up-regulates activity,binding,...,NaN,NaN,NaN,31958465,YES,NaN,miannu,"Milk fat globule-EGF factor 8 (MFGE8), a prote...",0.524,SIGNOR-260644


In [4]:

## check all unique combinations between subject & object
from itertools import combinations

# Get all combinations of 3 column names
col_combos = ['subject_category', 'object_category', 'EFFECT']

# Get unique combinations and their counts
combo_counts = source_df.groupby(col_combos).size().reset_index(name='count')

print(combo_counts)

      subject_category    object_category  \
0    BiologicalProcess  BiologicalProcess   
1    BiologicalProcess  BiologicalProcess   
2    BiologicalProcess            complex   
3    BiologicalProcess            complex   
4    BiologicalProcess            complex   
..                 ...                ...   
251      smallmolecule      proteinfamily   
252      smallmolecule      smallmolecule   
253      smallmolecule      smallmolecule   
254      smallmolecule      smallmolecule   
255      smallmolecule      smallmolecule   

                                  EFFECT  count  
0                         down-regulates      3  
1                           up-regulates     25  
2                         down-regulates      2  
3                down-regulates activity      1  
4                           up-regulates      5  
..                                   ...    ...  
251  up-regulates quantity by expression      1  
252                       down-regulates      1  
253      

In [5]:
## Define the output path for node & edge files after formatting
download_path_node_file = f'/Users/Weiqi0/ISB_working/Ilya_lab/Translator/Pharmagenomics_KG/files/temp/SIGNOR_source_combos.tsv'

## download both node and edge files
## Download the result df
## disable download for testing
combo_counts.to_csv(download_path_node_file, sep ='\t', index=False)